## Post-block Assignment 1

In [ ]:
# !pip install -Uqq plotly

In [22]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import plotly.express as px
import warnings
warnings.filterwarnings("ignore")
PLOTLY_THEME = "plotly_dark"

## 1. Read the data

In [23]:
def read_data(df):

    df = pd.read_csv(df)
    df.columns = df.columns.str.lower().str.replace(' ', '_') # Ensure all columns follow rules of declaring variables in coding
    df = df.drop(axis=1, columns=['unnamed:_0', 'index'])
    return df

df = read_data(df = "data/DataScientist (1).csv")
print(df.shape)
df.sample(5)

(3909, 15)


,job_title,salary_estimate,job_description,rating,company_name,location,headquarters,size,founded,type_of_ownership,industry,sector,revenue,competitors,easy_apply
2561,Reporting Data Analyst,$49K-$78K (Glassdoor est.),Power BI and SSIS AzureSQL is a big plus Requi...,-1.0,Shiro Technologies,"Dallas, TX","Dallas, TX",1 to 50 employees,-1,Company - Private,-1,-1,Less than $1 million (USD),-1,-1
1428,Big Data Engineer with Cloudera,$92K-$144K (Glassdoor est.),"Hi, Big Data Engineer w Cloudera Location Phoe...",5.0,Nucleusteq\n5.0,"Phoenix, AZ","Phoenix, AZ",201 to 500 employees,2018,Company - Private,IT Services,Information Technology,$10 to $25 million (USD),-1,-1
3906,Security Analytics Data Engineer,$55K-$112K (Glassdoor est.),Job DescriptionThe Security Analytics Data Eng...,3.8,"PDS Tech, Inc.\n3.8","Dublin, OH","Irving, TX",5001 to 10000 employees,1977,Company - Private,Staffing & Outsourcing,Business Services,$100 to $500 million (USD),-1,-1
2626,Machine Learning Engineer,$85K-$159K (Glassdoor est.),"Req ID: 90379\n\nAt NTT DATA Services, we know...",3.4,NTT DATA\n3.4,"Irving, TX","Tokyo, Japan",10000+ employees,1967,Company - Public,IT Services,Information Technology,$10+ billion (USD),"Capgemini, Accenture, Deloitte",-1
695,Quantitative Analyst,$46K-$80K (Glassdoor est.),Position Overview\n\nWe are seeking a Quantita...,3.8,Magnetar Capital\n3.8,"Evanston, IL","Evanston, IL",201 to 500 employees,2005,Company - Private,Investment Banking & Asset Management,Finance,Unknown / Non-Applicable,"AQR Capital Management, Citadel, Two Sigma",-1


--- 

## 2. Data Preprocessing
- This entails data cleaning, extracting features, imputing missing values, remove noise etc

In [24]:
def clean_salary_estimates(df, column_name='salary_estimate'):
    
    df[column_name] = df[column_name].str.replace(r'\(.*\)', '', regex=True)  # Clean the salary_estimate column by removing extra text in parentheses,remove extra text

    # Remove '$' and 'K', then split by '-'
    df[['min_salary_estimate', 'max_salary_estimate']] = df[column_name].str.replace(r'[^\d-]', '', regex=True) \
                                                     .str.split('-', expand=True)
    # Replace 'K' with '000' to convert to actual values
    df['min_salary_estimate'] = df['min_salary_estimate'].str.replace('K', '').astype(int) * 1000
    df['max_salary_estimate'] = df['max_salary_estimate'].str.replace('K', '').astype(int) * 1000

    return df

In [25]:
def split_revenue(revenue):
    if "to" in revenue:
        return revenue.replace(" (USD)", "").split(" to ")
    elif "Less than" in revenue:
        return ["0", revenue.replace("Less than ", "").replace(" (USD)", "")]
    elif "Unknown" in revenue or revenue == "-1":
        return [None, None]
    elif "$10+ billion" in revenue:
        return ["$10 billion", None]  # Assuming $10+ billion is minimum at 10B
    else:
        return [None, None]

# Apply function to create min_revenue and max_revenue columns
df[["min_revenue", "max_revenue"]] = pd.DataFrame(df["revenue"].apply(split_revenue).tolist(), index=df.index)

In [26]:
def classify_size(size):
    """Classify company size based on employee count."""
    if isinstance(size, str):  # Ensure it's a string
        size = size.replace(" employees", "").strip()
        
        if size == "-1":
            return "Unknown"
        elif "10000+" in size:
            return "Large"
        elif " to " in size:
            min_size, max_size = map(int, size.split(" to "))
            if max_size <= 200:
                return "Small"
            elif max_size <= 1000:
                return "Medium"
            else:
                return "Large"
            
        elif size.isdigit():  # Single number case
            num = int(size)
            if num <= 200:
                return "Small"
            elif num <= 1000:
                return "Medium"
            else:
                return "Large"
    
    return "Unknown"

# Apply company size classification
df["company_size_category"] = df["size"].apply(classify_size)


In [27]:
df = df[~df['sector'].str.contains(r'\d', na=False)] # Remove numbers on feature sector
df = clean_salary_estimates(df)
df.sample(4)

,job_title,salary_estimate,job_description,rating,company_name,location,headquarters,size,founded,type_of_ownership,industry,sector,revenue,competitors,easy_apply,min_revenue,max_revenue,company_size_category,min_salary_estimate,max_salary_estimate
3872,IGM - Post Doctoral Scientist - Chaudhari Lab,$39K-$86K,JOB POSTING - Post Doctoral Scientist IGM\nFul...,3.7,Nationwide Children's Hospital\n3.7,"Columbus, OH","Columbus, OH",10000+ employees,1892,Nonprofit Organization,Health Care Services & Hospitals,Health Care,$1 to $2 billion (USD),-1,-1,$1,$2 billion,Large,39000,86000
250,Senior Data Scientist,$133K-$171K,Description\n\nThe Senior Data Scientist uses ...,3.6,Humana\n3.6,"Jersey City, NJ","Louisville, KY",10000+ employees,1961,Company - Public,Insurance Carriers,Insurance,$10+ billion (USD),"Cigna, Aetna, UnitedHealth Group",-1,$10 billion,None,Large,133000,171000
1537,Big Data Engineer,$84K-$156K,2 to 3 years' experience designing and develop...,4.0,Lorven Technologies Inc\n4.0,"Phoenix, AZ","Plainsboro, NJ",1 to 50 employees,-1,Company - Private,Accounting,Accounting & Legal,Less than $1 million (USD),-1,-1,0,$1 million,Small,84000,156000
3863,Data Engineer (Enterprise Content Management w...,$39K-$86K,"Hello, We have an opening for Data Engineer (E...",4.0,JASStek Inc.\n4.0,"Columbus, OH","Dublin, OH",1 to 50 employees,2004,Company - Private,Computer Hardware & Software,Information Technology,$1 to $5 million (USD),-1,-1,$1,$5 million,Small,39000,86000


---

<div style="text-align:center; font-size:30px; font-weight:bold;"> Derive insights of strategic value</div>

## Salary By Sector

In [28]:
fig = px.box(data_frame=df, x = "sector", y = "min_salary_estimate")
fig.show()

### Number of job posting by location

In [29]:
roles_by_sectors = df.groupby(['sector'])[['job_title']]\
                     .count()\
                     .reset_index()\
                     .sort_values(by='job_title', ascending=False)\
                     .head(10)
                     
roles_by_sectors = roles_by_sectors.rename(columns={'job_title': 'num_of_job_posting'})

fig = px.bar(roles_by_sectors, x = "sector", y = "num_of_job_posting", template=PLOTLY_THEME, title="Number of Job Posting by Sector")
fig.update_layout(title_x = 0.5)
fig.show()

### NB Provide insigths on location based industries and support the statement using references/articles to be in a poster

---

In [30]:
roles_by_company_size = df.groupby(['company_size_category'])[['job_title']]\
                          .count()\
                          .reset_index()\
                          
roles_by_company_size = roles_by_company_size.rename(columns={"job_title": "num_of_job_posting"})                          
                          
roles_by_company_size.head()

,company_size_category,num_of_job_posting
0,Large,1738
1,Medium,696
2,Small,914
3,Unknown,15


In [ ]:
fig = px.pie(roles_by_company_size, values='num_of_job_posting', names="company_size_category", template=PLOTLY_THEME)
fig.show()

##### Provide insights
---

#### Get the avg min salary estimate by sector and location

In [34]:

avg_min_salary_by_sector = df.groupby(['sector','location'])[['min_salary_estimate']]\
                             .mean()\
                             .reset_index()\
                             .sort_values(by=['min_salary_estimate'], ascending=False)\
                             .round(2)
	

#print(avg_min_salary_by_sector.head(10)) # Top 10 of most paying sectors by sector and location

fig = px.bar(data_frame=avg_min_salary_by_sector.head(10).sort_values(by='sector', ascending=False),
              x = "sector",
              y = "min_salary_estimate",
              color="location",
              title="Average min salary by sector and location", 
              template=PLOTLY_THEME)
fig.update_layout(title_x=0.5)
fig.show()

## 1. High-Paying Sectors and Locations
- Finance stands out as the sector with the highest minimum salary estimates, particularly in Santa Clara, CA.
- Information Technology (IT), a major sector for data science roles, shows a strong salary offering, particularly in Redwood City, CA.
- Real Estate, Health Care, and Business Services also offer competitive salaries, indicating potential demand for data analytics and AI-driven decision-making.


## 2. Data Science Job Monetization Opportunity
- The Finance and IT sectors are particularly lucrative for data science professionals, making them prime targets for recruitment solutions.
- Santa Clara and Redwood City, CA, seem to be key locations offering high-paying jobs, meaning job postings from these locations could be prioritized for insights and monetization.

---

#### Demand for Data Science Roles by Company Size

In [35]:
demand_by_company_size = df.groupby(['industry', 'company_size_category'])[['job_title']]\
                      .count()\
                      .reset_index()\
                      .sort_values(by = "job_title", ascending=False)

demand_by_company_size.head(10)

,industry,company_size_category,job_title
20,Biotech & Pharmaceuticals,Large,173
105,IT Services,Small,171
103,IT Services,Large,160
95,Health Care Services & Hospitals,Large,151
115,Internet,Large,148
104,IT Services,Medium,140
37,Computer Hardware & Software,Large,131
179,Staffing & Outsourcing,Small,121
177,Staffing & Outsourcing,Large,104
39,Computer Hardware & Software,Small,102


In [36]:
#Salary Trends Based on Company Size

avg_min_salary_by_comany_size = df.groupby(['company_size_category'])[['max_salary_estimate']]\
                                  .median()\
                                  .reset_index()\
                                  .round(2)


avg_min_salary_by_comany_size

,company_size_category,max_salary_estimate
0,Large,136000.0
1,Medium,122000.0
2,Small,139000.0
3,Unknown,149000.0


In [37]:
fig = px.bar(data_frame=avg_min_salary_by_comany_size,
             x='company_size_category', 
             y='max_salary_estimate')
fig.show()

### How many Job Posting by Industry and Location?

In [38]:
roles_by_industry = df.groupby(['industry', 'company_size_category'])[['job_title']]\
                      .count()\
                      .reset_index()\
                      .sort_values(by = "job_title", ascending=False)

In [39]:
fig = px.bar(data_frame=roles_by_industry.head(10), x = "industry",
                                                    y = "job_title",
                                                    color = "company_size_category",
                                                    title="Number of Job Posting by location and industry", 
                                                    template=PLOTLY_THEME)
fig.update_layout(title_x=0.5)
fig.show()